1 - Снять видео на камеру (любую, можно телефона). Выполнить кручение видео со скоростью 10 градусов в секунду относительно центра (в любую сторону). <br>
2 - Видео после выполнения п.1 и вернуть его в исходное состояние. 

Для получения исходного видео из повернутого нам нужно сохранить информацию пикселей, которые не попадут в кадр после поворота. Для этого будем поворачивать и масшабировать, чтобы кадр полностью поместился после поворота. Масштаб расчитаем исходя из математики и высоты повернутого кадра для каждого угла поворота. Его наибольшая занимаемая высота рассчитываетя как рассояние между верхним и нижним углами. При обратных поворотах применяем те же коэффициенты масштабирования, возвращаем кадр до исходных размеров.

In [4]:
from IPython.display import HTML
import numpy as np
import ffmpeg
import cv2


SOURCE_VIDEO_PATH = 'videos/source_video.avi'
TIME_SEC = 10

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)

out_video = cv2.VideoWriter(SOURCE_VIDEO_PATH, fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
                            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
frame_counter = 0
while cap.isOpened():
    success, frame = cap.read()

    # count frames and break after all frames in 10 sec
    if not success or fps * TIME_SEC < frame_counter:
        break

    out_video.write(frame)
    frame_counter += 1

    # cv2.imshow('Camera', frame)
    # if cv2.waitKey(1) == ord('q'):
    #     break

out_video.release()
cap.release()
cv2.destroyAllWindows()


In [1]:
# function to calculate scale for frame in each rotation angle
def get_scale(angle_deg: float):
    global H, D, angleWD_rad

    angle_rad = np.deg2rad(angle_deg) % (2 * np.pi)
    if angle_rad <= np.pi / 2:
        angle_rad = angle_rad + angleWD_rad
    elif angle_rad <= np.pi:
        angle_rad = np.pi - angle_rad + angleWD_rad
    elif angle_rad <= 3 * np.pi / 2:
        angle_rad = angle_rad + angleWD_rad - np.pi
    else:
        angle_rad = 2 * np.pi - angle_rad + angleWD_rad
    scale = H / (D * np.sin(angle_rad))

    return scale


In [2]:
# function to rotate video
def video_rotation(source_video_path: str, out_video_path: str, angle_delta: float, 
                   is_inverse_roration: bool = False):
    cap = cv2.VideoCapture(source_video_path)

    # calculate global variables as frame width, height, diagonal size and angle between width and diagonal 
    # to calculate scale than
    global W, H, D, angleWD_rad
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    D = np.sqrt(H ** 2 + W ** 2)
    angleWD_rad = np.acos(W / D)

    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_vriter = cv2.VideoWriter(out_video_path, fourcc, fps, (W, H))

    center_point = (W // 2, H // 2)
    angle_deg = -angle_delta
    frames_counter = 0

    # use flag tp rotate in reverse direction with inverse transrofm matrix
    flag = cv2.WARP_INVERSE_MAP if is_inverse_roration else cv2.INTER_LINEAR

    while True:
        success, frame = cap.read()
        if not success:
            break

        # we need to rotate frame every second, than we wait fps-count frames to recalculate rotation angle
        if frames_counter % fps == 0:
            angle_deg = frames_counter // fps * angle_delta
            scale = get_scale(angle_deg)
            rotation_matrix = cv2.getRotationMatrix2D(center=center_point, angle=angle_deg, 
                                                      scale=scale)

        rotated_frame = cv2.warpAffine(src=frame, M=rotation_matrix, dsize=(W, H), flags=flag)
        video_vriter.write(rotated_frame)
        frames_counter += 1
    video_vriter.release()
    cap.release()


In [6]:
ROTATED_VIDEO_PATH = 'videos/rotated_video.avi'
INVERSE_VIDEO_PATH = 'videos/inverse_video.avi'

# make rotation video from source and original video from roteted
video_rotation(SOURCE_VIDEO_PATH, ROTATED_VIDEO_PATH, 10)
video_rotation(ROTATED_VIDEO_PATH, INVERSE_VIDEO_PATH, 10, True)    # use True-flag to rotate video in inverse direction


3 - Сконвертируйте видео в mp4 при помощи ffmpeg

In [8]:
# function to convert video from .avi to .mp4
def convert_to_mp4(video_path: str):
    video_mp4_path = '.'.join(video_path.split('.')[:-1] + ['mp4'])
    stream = ffmpeg.input(video_path)
    stream.output(video_mp4_path).run()

    return video_mp4_path


In [9]:
# convert videos to .mp4
SOURCE_VIDEO_MP4_PATH = convert_to_mp4(SOURCE_VIDEO_PATH)
ROTATED_VIDEO_MP4_PATH = convert_to_mp4(ROTATED_VIDEO_PATH)
INVERSE_VIDEO_MP4_PATH = convert_to_mp4(INVERSE_VIDEO_PATH)


In [10]:
# display videos through HTML
HTML(f"""
    <video width="{W}" height="{H}" controls>
        <source src={SOURCE_VIDEO_MP4_PATH} type="video/mp4">
    </video>
    <video width="{W}" height="{H}" controls>
        <source src={ROTATED_VIDEO_MP4_PATH} type="video/mp4">
    </video>
    <video width="{W}" height="{H}" controls>
        <source src={INVERSE_VIDEO_MP4_PATH} type="video/mp4">
    </video>
""")


Ссылки на видео:<br>
<a href="https://drive.google.com/file/d/1mCUGTtnuD_6GeGoXvOSeGCK2acjWsLDk/view?usp=drive_link">Исходное видео</a><br>
<a href="https://drive.google.com/file/d/1inpb-rBgfEWvXZchqYPB24BeX5khm785/view?usp=drive_link">Повернутое видео</a><br>
<a href="https://drive.google.com/file/d/18t2PibOR9jKz0AjuKc0u3x0La_tvGDAz/view?usp=drive_link">Видео после возвращения в исходное состояние</a>